In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html

In [2]:
import plotly.graph_objs as go
from dash.dependencies import Input, Output
import pandas as pd
from rdkit import Chem
from rdkit.Chem.Draw import MolsToGridImage
import base64
from io import BytesIO

Reading in the tsne df for the HBD generated previously

In [3]:
df = pd.read_csv("hbd_tsne_df_w_cid.csv")

In [4]:
df

,X,Y,rank,CID,SMILES
0,-34.911650,29.599580,1,1176,C(=O)(N)N
1,-63.656920,16.410173,2,522583,CC(N)O
2,-62.669167,16.095873,3,71309146,CC(C)O
3,-62.668095,16.095135,4,12213345,CC(C)O
4,-32.310570,31.257702,5,11182567,CC(=O)O
5,-57.653362,-1.751240,6,2733137,C(CO)O
6,-66.255806,12.919889,7,1030,CC(CO)O
7,-62.669556,16.094725,8,12213346,CC(C)O
8,-57.851570,-0.474372,9,10442,C(CO)CO
9,-66.255646,12.919264,10,439846,CC(CO)O


Generate scatter plot

In [5]:
graph_component = dcc.Graph(
    id='tsne',
    config={'displayModeBar': False},
    figure={
        'data': [
            go.Scattergl(
                x=df.X,
                y=df.Y,
                mode='markers',
                opacity=0.7,
                marker={
                    'size': 5,
                    'color': 'orange',
                    'line': {'width': 0.5, 'color': 'white'}
                },
                name="Decoy"
            ),
            
            
        ],
        'layout': go.Layout(
            height=400,
            xaxis={'title': 'X'},
            yaxis={'title': 'Y'},
            margin={'l': 40, 'b': 40, 't': 10, 'r': 10},
            legend={'x': 1, 'y': 1},
            hovermode=False,
            dragmode='select'
        )
    }
)

Generate component that will hold structure image from rdkit.

In [6]:
image_component = html.Img(id="structure-image")

Creat the dash application object

In [7]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

The application's layout is initialized with HTML divs containing the two objects we created above. 

In [8]:
app.layout = html.Div([
    html.Div([graph_component]),
    html.Div([image_component])
])


We will define a callback that is called whenever points are selected in the plot. 

In [9]:
@app.callback(
    Output('structure-image', 'src'),
    [Input('tsne', 'selectedData')])

SyntaxError: unexpected EOF while parsing (<ipython-input-9-05d5daabb3df>, line 3)